In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import torch
from gpt import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 128, # Shortened context length (orig: 1024)
    "emb_dim": 256,        # Embedding dimension
    "n_heads": 4,         # Number of attention heads
    "n_layers": 4,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

In [4]:
import tiktoken
from gpt import generate_text_simple, create_dataloader_v1

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you mosquitoes popcorn appeaseAMES Lucy decide Pittsburgh Stainless utilizationRegistration


In [5]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-v1", split="train")


In [6]:
dataset['text']


['',
 ' = Valkyria Chronicles III = \n',
 '',
 ' Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving

In [7]:
txt_data = " ".join(dataset["text"])
# train_loader = create_dataloader_v1(txt_data, batch_size=4, max_length=256, stride=128)

train_ratio = 0.90
split_idx = int(train_ratio * len(txt_data))
train_data = txt_data[:split_idx]
val_data = txt_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=4,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=4,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [8]:
for batch_idx, (input_batch, target_batch) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}")
    print("Input batch (tokens):", input_batch)
    print("Target batch (tokens):", target_batch)
    break  # Only printing the first batch as an example

Batch 1
Input batch (tokens): tensor([[ 2422,  9964,  1028,  4849, 17072,   772,   981,   991,   739,  1886,
          2162,  3977, 12444,   284, 21194,   625,  9964,   810, 26402,   373,
           407,  1682,   287,  3877,   837, 41109,   284,  1277,   465, 13463,
          3371,   262, 39785,  5822,  2138,   621, 34618, 22576,   764,  3977,
           705,    82,  2106,   460,   307,  1775,   355,   281,  8453,   544,
           837,   257, 16716,  3761,   837,   329,   262, 13239,   837,   290,
           517,  5734,   329, 26402,   705,    82,  3896,   764,  2750,   262,
          1367,  2154,    82,   290,  1367,  1795,    82,   837,  8830, 20006,
           547, 18416,   284,  1104,   262, 13239,   837, 11476,   780,   340,
           373,  1290,  1497,   290,   612,   547,   517, 12273,  4786,   287,
          2031,   837,   475,   635,   780,   443,  1676,  1837,   373,  3221,
          3177, 11871,  9837,   764,   220,   198,   220,  3977,   373, 20524,
         21925,  1028,

In [28]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [11]:

# # Note:
# # Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# # which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# # However, the resulting loss values may be slightly different.

# #if torch.cuda.is_available():
# #    device = torch.device("cuda")
# #elif torch.backends.mps.is_available():
# #    device = torch.device("mps")
# #else:
# #    device = torch.device("cpu")
# #
# # print(f"Using {device} device.")


# model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


# torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

# with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
#     train_loss = calc_loss_loader(train_loader, model, device)
#     val_loss = calc_loss_loader(val_loader, model, device)

# print("Training loss:", train_loss)
# print("Validation loss:", val_loss)


In [36]:
import logging
import deepspeed
from tqdm import tqdm  # tqdm을 함수처럼 사용

# Setup logging configuration
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s", 
    level=logging.INFO,  # You can adjust the level to DEBUG, INFO, WARNING, etc.
    handlers=[
        logging.FileHandler("training_log.txt"),  # Logs will be saved to this file
        logging.StreamHandler()  # Also log to console
    ]
)

# Now use logging instead of print
logger = logging.getLogger(__name__)

In [45]:
import deepspeed


def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer, deepspeed_config):

    # Initialize DeepSpeed
    model, optimizer, _, _ = deepspeed.initialize(args=None, model=model, optimizer=optimizer, config_params=deepspeed_config)

    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        logger.info(f"Starting Epoch {epoch+1}...")

        for input_batch, target_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}", unit="batch"):
            input_batch = input_batch.to(device)
            target_batch = target_batch.to(device)

            optimizer.zero_grad()  # Reset loss gradients from previous batch iteration

            # Compute the loss
            loss = calc_loss_batch(input_batch, target_batch, model, device)

            # print(loss)
            # print(f"Loss type: {loss.dtype}")
            # print(f"Loss scale type: {model.loss_scale.dtype}")
            # if not isinstance(loss, torch.Tensor):
            #     loss = torch.tensor(loss).to(device)


            # Backward pass and step the optimizer
            model.backward(loss)  # Backward pass with DeepSpeed
            model.step()  # Step optimizer using DeepSpeed

            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter
                )
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                logger.info(f"Epoch {epoch+1} (Step {global_step:06d}): "
                            f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
                
                
        logger.info(f"Epoch {epoch+1} completed. Generating a sample...")
        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format
    logger.info(f"Generated Text: {decoded_text}")  # Log the generated text
    model.train()

In [46]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

train_losses, track_tokens_seen = train_model_simple(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=3,
    eval_freq=1000,
    eval_iter=10,
    start_context="Every effort moves you",
    tokenizer=tokenizer,
    deepspeed_config="./ds_config.json"
)

[2025-04-02 19:35:32,917] [INFO] [logging.py:107:log_dist] [Rank 0] DeepSpeed info: version=0.16.5, git-hash=unknown, git-branch=unknown
[2025-04-02 19:35:32,919] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1
[2025-04-02 19:35:32,924] [INFO] [logging.py:107:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-04-02 19:35:32,925] [INFO] [logging.py:107:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2025-04-02 19:35:32,925] [INFO] [logging.py:107:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-04-02 19:35:32,926] [INFO] [logging.py:107:log_dist] [Rank 0] DeepSpeed Basic Optimizer = AdamW
[2025-04-02 19:35:32,926] [INFO] [utils.py:59:is_zero_supported_optimizer] Checking ZeRO support for optimizer=AdamW type=<class 'torch.optim.adamw.AdamW'>
[2025-04-02 19:35:32,926] [INFO] [logging.py:107:log_dist] [Rank 0] Creating torch.float16 ZeRO stage 2 optimizer
[2025-04-02 19:35:32,926] [INFO] [stage_1_a

2025-04-02 19:35:35,634 - INFO - Starting Epoch 1...
Epoch 1:   0%|          | 0/210244 [00:00<?, ?batch/s]2025-04-02 19:35:35,733 - INFO - Epoch 1 (Step 000000): Train loss 10.962, Val loss 10.964


[2025-04-02 19:35:35,746] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 4294967296, reducing to 2147483648


Epoch 1:   0%|          | 2/210244 [00:00<3:15:37, 17.91batch/s]

[2025-04-02 19:35:35,775] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 2147483648, reducing to 1073741824
[2025-04-02 19:35:35,794] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 1073741824, reducing to 536870912
[2025-04-02 19:35:35,814] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 536870912, reducing to 268435456
[2025-04-02 19:35:35,833] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 268435456, reducing to 134217728
[2025-04-02 19:35:35,852] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 134217728, reducing to 67108864


...

Epoch 3:  99%|█████████▉| 208697/210244 [45:10<00:19, 80.22batch/s]

[2025-04-02 21:51:49,367] [INFO] [loss_scaler.py:183:update_scale] [deepspeed] OVERFLOW! Rank 0 Skipping step. Attempted loss scale: 8192, reducing to 4096


Epoch 3: 100%|██████████| 210244/210244 [45:30<00:00, 76.99batch/s]
2025-04-02 21:52:09,905 - INFO - Epoch 3 completed. Generating a sample...
2025-04-02 21:52:09,979 - INFO - Generated Text: Every effort moves you out of the room . " 
   = = = = = = 
   The song was released on the iTunes Store on September 2 , 2008 , and was released on September 2 , 2008 . It was released on September 2 ,


Every effort moves you out of the room . "     = = = = = =     The song was released on the iTunes Store on September 2 , 2008 , and was released on September 2 , 2008 . It was released on September 2 ,


ValueError: too many values to unpack (expected 2)

In [12]:
from transformers import Trainer, TrainingArguments


# 6. Training Arguments
training_args = TrainingArguments(
    output_dir="./sllm_output",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="no",
    save_strategy="no",
    logging_steps=10,
    deepspeed="./ds_config.json",  # <- 여전히 사용 가능
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
    # ddp_backend=None
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

/home/jiyoon/miniconda3/envs/LLMs/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3213514/2491510290.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# import os
# os.environ["USE_MPI"] = "0"



trainer.train()  # Now, trainer.train() will handle the DataLoader and training.

[2025-04-01 23:59:57,902] [WARNING] [lr_schedules.py:683:get_lr] Attempting to get learning rate from scheduler before it has started


TypeError: vars() argument must have __dict__ attribute